# Automação para preenchimento de páginas

### Pacotes
Manipular dados dos dos municípios, timer para funcionamento do código e manipulação de informações contida na área de transferência do computador.

In [37]:
import pandas as pd     #manipular dados
from time import sleep  #timer para realizar o captcha
import pyperclip        #usar conteudo na area de transferencia (copia e cola)

### pacotes para utilizar o navegador e mouse

In [1]:
from selenium import webdriver                  #utilizar o navegador
from selenium.webdriver.common.by import By     #procurar elementos pelo nome
from selenium.webdriver.common.keys import Keys #utilizar teclas não-alfanuméricas
import pyautogui                                #controlar mouse e teclado

### Lista com todos municipios

In [75]:
MunicipiosParana = pd.read_excel('cidadesParana.xlsx') #base com municipios do parana

In [ ]:
MunicipiosParana.head() #A base possui o nome e código de referência no IPARDES


### Script para fazer login como adminstrador no site e looping de preenchimento

In [ ]:
############ Login como administrador do site ############

firefox = webdriver.Firefox(executable_path='geckodriver.exe')          #manipula o firefox
firefox.get("http://www.agepar.pr.gov.br/user")                         #abre pagina de login da agepar
firefox.find_element_by_name('name').send_keys('')                      #digita usuário
firefox.find_element_by_name('pass').send_keys('')                      #digita senha
sleep(5)                                                                #nesse momento, responder a conta do captcha...
firefox.find_element_by_xpath('//*[@id="edit-submit"]').click()         #aperta enter

############ Loop para preenchimento das páginas ##############

for i in range(347):                    #Percorre cada uma das linhas da base
    firefox.get(f"http://www.agepar.pr.gov.br/Endereco/{MunicipiosParana['Municipios'][i]}")                                #acessa página do municipio
    firefox.find_element_by_xpath('/html/body/div[1]/div[2]/section/div/section/div/div/div/div/nav/ul/li[2]/a').click()    #seleciona a aba de edição
    sleep(5)                            #espera a pagina carregar todos os elementos
    pyautogui.click(x=-1263, y=985)     #clica no botão que ativa o modo código-fonte
    sleep(1)                            #espera a conversão de texto em html
    pyautogui.click(x=-1058, y=1048)    #clica na caixa de preenchimento de texto 
    pyautogui.hotkey('ctrl', 'a')       #atalho para selecionar todo o texto na caixa
    pyautogui.press('delete')           #deleta o texto (se houver)
    pyperclip.copy(f'''<div class="layoutmanager"> 
<div class="container-fluid layout-container">
<div class="layout-row row">
<div class="col-sm-12 layout-column">
<div class="layout-column-editable layout-column-one">
<p>Agência Reguladora de Serviços Públicos Delegados do Paraná (Agepar)</p>

<p>Clique no link e baixe as informações sobre a sua cidade:</p>

<p><span class="fontawesome-icon-inline"><span class="fontawesome-icon-inline"></span></span><a href="http://www.agepar.pr.gov.br/sites/default/arquivos_restritos/files/documento/2021-06/{MunicipiosParana['Municipios'][i]}.pdf" target="_blank"><span class="fontawesome-icon-inline"><span class="fa-2x fa-faucet fas"></span>&nbsp;</span><strong>Serviços Públicos de Saneamento</strong></a></p>

<p><a href="http://www.ipardes.gov.br/cadernos/MontaCadPdf1.php?Municipio={MunicipiosParana['Codigo'][i]}&amp;btOk=ok" target="_blank"><span class="fontawesome-icon-inline"><span class="fa-2x fa-book-open fas"></span>&nbsp;</span><strong>Caderno Estatístico do Município</strong></a></p>
</div>
</div>
</div>
</div>
</div>

<hr />
<p>Prefeitura</p>''')   #copiar na area de transferencia todo o código html com a cidade e código de referência.
    pyautogui.hotkey("ctrl", "v")                                   #comando para colar texto na caixa
    firefox.find_element_by_xpath('//*[@id="edit-submit"]').click() #salva a edição 

firefox.close()         #quando o processo termina o navegador será fechado
print('Todos as páginas foram preenchidas')                         #mensagem de conclusão
